In [4]:
import yfinance as yf
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.model_selection import train_test_split

# Defining the stock values and downloading data from yahoo finance
stock_symbol = "GOOGL"
start_date = "2022-01-01"
end_date = "2023-01-01"
data = yf.download(stock_symbol, start=start_date, end=end_date)

print(data)

# Data Preprocessing using MinMaxScaler
def preprocess_data(data):
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(data[['Close']].values)

    return scaled_data, scaler

scaled_data, data_scaler = preprocess_data(data)

# Sequence-to-Sequence Transformation
def create_sequences(data, sequence_length):
    X, y = [], []
    for i in range(len(data) - sequence_length):
        X.append(data[i:i+sequence_length])
        y.append(data[i+sequence_length])
    return np.array(X), np.array(y)

sequence_length = 20
X, y = create_sequences(scaled_data, sequence_length)

# Split the Data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Model Building using Long Short Term Memory for hidden and Hyperparameter Tuning
model = Sequential()
model.add(LSTM(units=50, activation='tanh', input_shape=(sequence_length, 1)))
model.add(Dense(1, activation='linear'))
model.compile(optimizer='adam', loss='mean_squared_error')

# Model Training
epochs = 50
batch_size = 32
history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size)

# Model Evaluation
y_pred = model.predict(X_test)
y_pred = data_scaler.inverse_transform(y_pred)
y_test = data_scaler.inverse_transform(y_test)

mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error on Test Data: {mse}")


[*********************100%%**********************]  1 of 1 completed
                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2022-01-03  145.054993  145.850998  143.712997  144.991501  144.991501   
2022-01-04  145.395996  146.485001  143.716507  144.399506  144.399506   
2022-01-05  144.419998  144.499496  137.688004  137.774994  137.774994   
2022-01-06  136.998505  139.940002  136.558502  137.747498  137.747498   
2022-01-07  138.145493  138.448502  135.766495  137.016998  137.016998   
...                ...         ...         ...         ...         ...   
2022-12-23   87.110001   89.550003   87.070000   89.230003   89.230003   
2022-12-27   88.800003   88.940002   87.010002   87.389999   87.389999   
2022-12-28   86.980003   88.040001   85.940002   86.019997   86.019997   
2022-12-29   86.620003   88.849998   86.610001   88.449997   88.449997   
2022-12-30   86.980003   88.300003   86.570